<a href="https://colab.research.google.com/github/llDanieLll/BERT_FineTuned/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("CUDA available: ", torch.cuda.is_available())
!nvidia-smi

CUDA available:  True
Mon Sep  1 01:53:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------

In [2]:
!pip -q install transformers datasets evaluate accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 111.1 MB/s eta 0:00:00


In [4]:
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cuda


In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer

ckpt = "bert-base-uncased" # base BERT
ds = load_dataset("glue", "sst2") # SST-2 sentiment dataset
tok = AutoTokenizer.from_pretrained(ckpt)

def tokenize(batch):
  return tok(batch["sentence"], truncation=True, max_length=128)

ds_tok = ds.map(tokenize, batched=True, remove_columns=["sentence"])


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [15]:
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np, evaluate

collator = DataCollatorWithPadding(tokenizer=tok)
model = AutoModelForSequenceClassification.from_pretrained(ckpt, num_labels=2)

metric = evaluate.load("glue", "sst2")
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  return metric.compute(predictions=preds, references=labels)
args = TrainingArguments(
    output_dir="bert-sst2-t4",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    do_eval=True,
    save_steps=500,
    eval_steps=500,
    report_to="none",
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["validation"],
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1801559239.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
train_result = trainer.train()
train_result

Step,Training Loss
50,0.581500
100,0.389500
150,0.344500
200,0.300500
250,0.296200
300,0.271500
350,0.308200
400,0.273700
450,0.290900
500,0.298500


TrainOutput(global_step=12630, training_loss=0.13879326071327674, metrics={'train_runtime': 1251.5444, 'train_samples_per_second': 161.438, 'train_steps_per_second': 10.092, 'total_flos': 3656008794765660.0, 'train_loss': 0.13879326071327674, 'epoch': 3.0})

In [17]:
metrics = trainer.evaluate()
metrics

{'eval_loss': 0.38151589035987854,
 'eval_accuracy': 0.9208715596330275,
 'eval_runtime': 0.7562,
 'eval_samples_per_second': 1153.155,
 'eval_steps_per_second': 37.028,
 'epoch': 3.0}

In [18]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=trainer.model, tokenizer=tok, device=0)
print(pipe("This movie was absolutely fantastic!"))
print(pipe("That was the worst meal I've had in years."))

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9999369382858276}]
[{'label': 'LABEL_0', 'score': 0.9998058676719666}]


In [19]:
save_dir = "bert-sst2-t4-best"
trainer.model.save_pretrained(save_dir)
tok.save_pretrained(save_dir)

('bert-sst2-t4-best/tokenizer_config.json',
 'bert-sst2-t4-best/special_tokens_map.json',
 'bert-sst2-t4-best/vocab.txt',
 'bert-sst2-t4-best/added_tokens.json',
 'bert-sst2-t4-best/tokenizer.json')

In [22]:
!ls /content

BERT_FineTuned	bert-sst2-t4  bert-sst2-t4-best  sample_data


In [21]:
!git config --global user.name llDanieLll
!git config --global user.email danielqiujiahao@gmail.com

# Clone your repo (replace with your repo URL)
!git clone https://github.com/llDanieLll/BERT_FineTuned.git
%cd BERT_FineTuned
!cp /content ./bert_sst2_finetune.ipynb

with open("requirements.txt","w") as f:
  f.write("transformers>=4.44\ndatasets>=2.20\nevaluate>=0.4\naccelerate>=0.33\ntorch\n")

!git add bert_sst2_finetune.ipynb requirements.txt
!git commit -m "Add Colab notebook and requirements"

!git push

Cloning into 'BERT_FineTuned'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.
/content/BERT_FineTuned/BERT_FineTuned
cp: cannot stat '/content/Training.ipynb': No such file or directory
fatal: pathspec 'bert_sst2_finetune.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	requirements.txt

nothing added to commit but untracked files present (use "git add" to track)
fatal: could not read Username for 'https://github.com': No such device or address
